# El problema de la dieta


El objetivo del problema de la dieta es **seleccionar comida que satisfaga requerimientos nutricionales diarios en un costo minimo.**

El problema puede ser formulado como de programacion lineal, para el cual **las restricciones limitan el numero de calorias y la cantidad de vitaminas y minerales, grasas, sodio y colesterol en la dieta.**



# Pranteamiento del problema 

El problema de la dieta puede ser modelado matematicamente como problema de programacion lineal usando el siguiente modelo.

## Conjuntos
- $F$ = conjunto de comidas
- $N$ = conjunto de nutrientes

## Parametros

- $c_i$ costo por porcion de comida $i, \forall i \in F$
- $a_{ij}$ = cantidad de nutrientes $j$ en la comida $i,\forall i \in F, \forall j \in N$
- $Nmin_j$ = minimo nivel de nutrientes $j, \forall i¡j \in N$
- $Nmax_j$ = maximum nivel de nutrientes $j, \forall j \in N$
- $V_i$ = el volumen por porcion de comida $i, \forall i \in F$
- $Vmax $ = maximum volumen de comida consumida.

## Variables
- $x_i$ = numero de porciones de comida $i$ a consumir

## Objetivo

Minimizar el costo total de comida 
$min \sum_{i \in F} c_ix_i$

## Restricciones (constraints)

Limite de nutrietnes consumidos por cada nutriente $j\in N$
$Nmin_j \leq \sum_{i \in F} a_{ij}x_i \leq Nmax_j, \forall j \in N$

Limite del volumen de comida consumida
$\sum_{i \in F} V_ix_i \leq Vmax$

Cotas de consumo inferior
$x_i\geq 0, \forall i \in F$

In [23]:
from pyomo.environ import *
infinity = float('inf')

model = AbstractModel()


In [24]:
# Foods
model.F = Set()
# Nutrients
model.N = Set()


In [25]:
# Cost of each food
model.c    = Param(model.F, within=PositiveReals)
# Amount of nutrient in each food
model.a    = Param(model.F, model.N, within=NonNegativeReals)
# Lower and upper bound on each nutrient
model.Nmin = Param(model.N, within=NonNegativeReals, default=0.0)
model.Nmax = Param(model.N, within=NonNegativeReals, default=infinity)
# Volume per serving of food
model.V    = Param(model.F, within=PositiveReals)
# Maximum volume of food consumed
model.Vmax = Param(within=PositiveReals)

In [26]:
# Number of servings consumed of each food
model.x = Var(model.F, within=NonNegativeIntegers)

In [27]:
# Minimize the cost of food that is consumed
def cost_rule(model):
    return sum(model.c[i]*model.x[i] for i in model.F)
model.cost = Objective(rule=cost_rule)

In [28]:
# Limit nutrient consumption for each nutrient
def nutrient_rule(model, j):
    value = sum(model.a[i,j]*model.x[i] for i in model.F)
    return model.Nmin[j] <= value <= model.Nmax[j]
model.nutrient_limit = Constraint(model.N, rule=nutrient_rule)

# Limit the volume of food consumed
def volume_rule(model):
    return sum(model.V[i]*model.x[i] for i in model.F) <= model.Vmax
model.volume = Constraint(rule=volume_rule)

In [29]:
!cat diet.dat

param:  F:                          c     V  :=
  "Cheeseburger"                 1.84   4.0  
  "Ham Sandwich"                 2.19   7.5  
  "Hamburger"                    1.84   3.5  
  "Fish Sandwich"                1.44   5.0  
  "Chicken Sandwich"             2.29   7.3  
  "Fries"                         .77   2.6  
  "Sausage Biscuit"              1.29   4.1  
  "Lowfat Milk"                   .60   8.0 
  "Orange Juice"                  .72  12.0 ;

param Vmax := 75.0;

param:  N:       Nmin   Nmax :=
        Cal      2000      .
        Carbo     350    375
        Protein    55      .
        VitA      100      .
        VitC      100      .
        Calc      100      .
        Iron      100      . ;

param a:
                               Cal  Carbo Protein   VitA   VitC  Calc  Iron :=
  "Cheeseburger"               510     34     28     15      6    30    20
  "Ham Sandwich"               370     35     24     15     10    20    20
  "Hamburger"                  500     42

In [30]:
from pyomo.environ import *

In [33]:
solver = SolverFactory('glpk')

instance = model.create_instance('diet.dat')
results = solver.solve(instance,tee=True)
results.write()
instance.solutions.load_from(results)

for v in instance.component_objects(Var, active=True):
    print ("Variable",v)
    varobject = getattr(instance, str(v))
    for index in varobject:
        print ("   ",index, varobject[index].value)

GLPSOL: GLPK LP/MIP Solver, v4.57
Parameter(s) specified in the command line:
 --write /var/folders/jp/5g65spjj28z4bxpxm9h_hsjh0000gn/T/tmp8a3jt0vb.glpk.raw
 --wglp /var/folders/jp/5g65spjj28z4bxpxm9h_hsjh0000gn/T/tmpa6vnn6_r.glpk.glp
 --cpxlp /var/folders/jp/5g65spjj28z4bxpxm9h_hsjh0000gn/T/tmp7zfikrbg.pyomo.lp
Reading problem data from '/var/folders/jp/5g65spjj28z4bxpxm9h_hsjh0000gn/T/tmp7zfikrbg.pyomo.lp'...
10 rows, 10 columns, 77 non-zeros
9 integer variables, none of which are binary
143 lines were read
Writing problem data to '/var/folders/jp/5g65spjj28z4bxpxm9h_hsjh0000gn/T/tmpa6vnn6_r.glpk.glp'...
129 lines were written
GLPK Integer Optimizer, v4.57
10 rows, 10 columns, 77 non-zeros
9 integer variables, none of which are binary
Preprocessing...
9 rows, 9 columns, 76 non-zeros
9 integer variables, none of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  5.100e+02  ratio =  5.100e+02
GM: min|aij| =  1.607e-01  max|aij| =  6.223e+00  ratio =  3.873e+01
EQ: min|a

In [35]:
model.create_instance?

In [16]:
from __future__ import division
from pyomo.environ import *

model = AbstractModel()

model.m = Param(within=NonNegativeIntegers)
model.n = Param(within=NonNegativeIntegers)

model.I = RangeSet(1, model.m)
model.J = RangeSet(1, model.n)

model.a = Param(model.I, model.J)
model.b = Param(model.I)
model.c = Param(model.J)

# the next line declares a variable indexed by the set J
model.x = Var(model.J, domain=NonNegativeReals)

def obj_expression(model):
    return summation(model.c, model.x)

model.OBJ = Objective(rule=obj_expression)

def ax_constraint_rule(model, i):
    # return the expression for the constraint for i
    return sum(model.a[i,j] * model.x[j] for j in model.J) >= model.b[i]

# the next line creates one constraint for each member of the set model.I
model.AxbConstraint = Constraint(model.I, rule=ax_constraint_rule)

In [ ]:
from pyomo.environ import *

In [38]:
#Carnes
Carne = {'CarneVaca':80, 'CarneCerdo':68}
Grasa = {'CarneVaca':.20, 'CarneCerdo':.32}
pCarne = {'CarneVaca':.8, 'CarneCerdo':.6}

limit = 100
items = list(sorted(Carne.keys()))

In [45]:
items,Carne["CarneVaca"]

(['CarneCerdo', 'CarneVaca'], 80)

In [37]:
# Create model
m = ConcreteModel()

# Variables
m.x = Var(items, within=Binary)

# Objective
m.value = Objective(expr=sum(Carne[i]*m.x[i] for i in items), sense=minimize)

# Constraint
m.weight = Constraint(expr=sum(Grasa[i]*m.x[i] for i in items) <= limit)


# Optimize
solver = SolverFactory('glpk')
status = solver.solve(m)

# Print the status of the solved LP
print("Status = %s" % status.solver.termination_condition)

# Print the value of the variables at the optimum
for i in items:
    print("%s = %f" % (m.x[i], value(m.x[i])))

# Print the value of the objective
print("Objective = %f" % value(m.value))

Status = optimal
x[hammer] = 1.000000
x[screwdriver] = 1.000000
x[towel] = 1.000000
x[wrench] = 1.000000
Objective = 28.000000


In [46]:


c=1.0
b=1.5
h=0.1
d = {1:15, 2:60, 3:72, 4:78, 5:82}

scenarios = range(1,6)

M = ConcreteModel()
M.x = Var(within=NonNegativeReals)
M.y = Var(scenarios)

M.c = ConstraintList()
for i in scenarios:
  M.c.add( M.y[i] >= (c-b)*M.x + b*d[i] )
  M.c.add( M.y[i] >= (c+h)*M.x - h*d[i] )

M.o = Objective(expr=sum(M.y[i] for i in scenarios)/5.0)

solver = SolverFactory('glpk')
status = solver.solve(M)

print("Status = %s" % status.solver.termination_condition)

print("%s = %f" % (M.x, value(M.x)))
for i in scenarios:
    print("%s = %f" % (M.y[i], value(M.y[i])))
print("Objective = %f" % value(M.o))

Status = optimal
x = 60.000000
y[1] = 64.500000
y[2] = 60.000000
y[3] = 78.000000
y[4] = 87.000000
y[5] = 93.000000
Objective = 76.500000
